In [ ]:
!pip install pythreejs
!pip install notebook
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user


In [ ]:
from pythreejs import *
from IPython.display import display

# Create a 3D sphere
sphere = Mesh(
    geometry=SphereGeometry(radius=1),
    material=MeshStandardMaterial(color='red')
)

# Rendering
width, height = 1600, 900

light = PointLight(position=[10, 10, 10], intensity=1.5)
scene = Scene(children=[sphere, light, AmbientLight(intensity=0.5)])


camera = PerspectiveCamera(position=[3, 3, 3], fov=50, aspect=width/height)
controller = OrbitControls(controlling=camera)


renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
display(renderer)


# Patricle Simulation

In [ ]:
from pythreejs import * 
from IPython.display import display

import ipywidgets as widgets
import numpy as np


def generate_subdivided_circle(radius=1.0, segments=16):
    """Generate vertices and triangular faces for a subdivided 2D circle."""
    vertices = []
    faces = []
    
    # Add the center vertex
    vertices.append([0.0, 0.0, 0.0])  # Center at (0, 0)
    
    # Generate boundary vertices
    for i in range(segments):
        theta = 2 * np.pi * i / segments
        x = radius * np.cos(theta)
        y = radius * np.sin(theta)
        vertices.append([x, y, 0.0])
    
    # Create faces (triangles)
    for i in range(segments):
        p1 = 0  # Center vertex
        p2 = i + 1
        p3 = (i + 1) % segments + 1  # Wrap around to the first boundary vertex
        faces.append([p1, p2, p3])
    
    return np.array(vertices, dtype=np.float32), np.array(faces, dtype=np.uint32)

# Generate sphere vertices and faces
radius, segments = 1.0, 64
vertices, faces = generate_subdivided_circle(radius=radius, segments=segments)
vertices[:, 1] += 3
# Create BufferGeometry using the vertices and faces
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(array=vertices, normalized=False),
    },
    index=BufferAttribute(array=faces.ravel(), normalized=False)
)

# Define material and mesh
material = MeshStandardMaterial(color='black', wireframe=True)
mesh = Mesh(geometry=geometry, material=material)



# Draw Ground
ground_vertices = np.array([
    [-10.0, 0.0, 0.0],  # Start point
    [10.0, 0.0, 0.0]    # End point
], dtype=np.float32)
ground_geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(ground_vertices, normalized=False)
    }
)
ground_material = LineBasicMaterial(color='red', linewidth=2)
line = Line(geometry=ground_geometry, material=ground_material)



# Rendering setup
width, height = 800, 600  # Smaller dimensions for 2D
aspect = width / height
scene = Scene(children=[mesh, line, AmbientLight(intensity=0.8)])
camera = OrthographicCamera(left=-5*aspect, right=5*aspect, top=5, bottom=-5, near=0.001, far=100, position=[0, 0, 10])
controller = OrbitControls(controlling=camera, enableRotate=False)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
renderer.background = '#FFFFFF'  # Optional: Set a white background for better 2D visibility


button = widgets.Button(desciption="RESET")  
output = widgets.Output()


def on_button_click(b):
    global vertices, faces, geometry
    with output:
        # Regenerate the circle
        vertices, faces = generate_subdivided_circle(radius=np.random.uniform(0.5, 2.0), segments=segments)
        vertices[:, 1] += 3  # Adjust position
        
        # Update geometry
        geometry.attributes['position'].array = vertices
        geometry.index.array = faces.ravel()
        geometry.attributes['position'].needsUpdate = True
        geometry.index.needsUpdate = True

button.on_click(on_button_click)

# Display the renderer and button
display(widgets.VBox([renderer, button, output]))

In [ ]:
import numpy as np
import time

# Constraints (for example, distance constraints between vertices)
def generate_constraints(vertices, faces):
    # Step 1: Extract unique edges from faces
    edges = set()
    for face in faces:
        # Add each edge of the triangle, sorted to avoid duplicates
        edges.add(tuple(sorted((face[0], face[1]))))
        edges.add(tuple(sorted((face[1], face[2]))))
        edges.add(tuple(sorted((face[2], face[0]))))
    
    # Step 2: Generate constraints
    constraints = []
    for edge in edges:
        v1, v2 = edge
        rest_length = np.linalg.norm(vertices[v1] - vertices[v2])
        constraints.append((v1, v2, rest_length))
    
    return constraints


def apply_external_force(velocities, gravity, time_step):
    v = velocities + inv_mass* gravity * time_step
    p = positions + v * time_step
    return p

def solve_constraints(positions, constraints, inv_mass, num_iterations):
    for _ in range(num_iterations):
        for (i, j, rest_length) in constraints:
            p1, p2 = positions[i], positions[j]
            delta = p2 - p1
            delta_length = np.linalg.norm(delta)
            if delta_length > 0:
                correction = (delta_length - rest_length) * delta / delta_length
                positions[i] += correction * inv_mass / 2
                positions[j] -= correction * inv_mass / 2
    return positions


# State
positions = vertices.copy()
velocities = np.zeros_like(positions)

# Physics parameters
mass = 1.0
inv_mass = 1.0 / mass
constraints = generate_constraints(vertices, faces)

# Simulation loop
# Simulation parameters
time_step = 1/30
num_iterations = 10
gravity = np.array([0, -9.81, 0])
for step in range(10000):  # Run for 100 steps
    positions = apply_external_force(velocities, gravity, time_step)
    positions = solve_constraints(positions, constraints, inv_mass, num_iterations)
    
    collide = np.where(positions[:, 1] < 0)
    positions[collide, 1] = 0
    # Update velocities based on new positions
    velocities = (positions - vertices) / time_step
    vertices = positions.copy()
    
    # Dynamic friction
    velocities[collide, 0] = velocities[collide, 0] * np.maximum(1 - 0.2*time_step*abs(velocities[collide, 0]), 0)
    # Static friction
    condition = np.abs(velocities[collide, 0]) < 0.9 * np.abs(velocities[collide, 1] + 1e-6)
    velocities[collide, 0] = np.where(condition, 0, velocities[collide, 0])
    
    # Render the updated positions (this part depends on your rendering setup)
    # For example, you can update the BufferGeometry with new positions
    geometry.attributes['position'].array = positions
    time.sleep(0.01)

# Rigid Body Simulation

In [1]:
from pythreejs import * 
from IPython.display import display

import ipywidgets as widgets
import numpy as np

Geometry


In [2]:
def draw_plane(origin=[0, 0, 0], normal=[0, 1, 0], size=10, color='gray', opacity=0.5):
    """Draw a plane with a given normal and size."""
    # Define the plane geometry
    plane_geometry = PlaneGeometry(width=size, height=size)
    plane_material = MeshStandardMaterial(
        color=color, 
        transparent=True, 
        opacity=opacity, 
        side='DoubleSide')
    plane = Mesh(geometry=plane_geometry, material=plane_material)
    plane.receiveShadow = True
    
    # Rotate the plane to the normal vector
    default_normal = np.array([0, 0, 1])  # PlaneGeometry default normal
    axis = np.cross(default_normal, normal)
    angle = np.arccos(np.dot(default_normal, normal))  # Angle between vectors
    
    if np.linalg.norm(axis) > 1e-6:  # Avoid division by zero if vectors are parallel
        axis = axis / np.linalg.norm(axis)
        plane.quaternion = list(np.append(axis * np.sin(angle / 2), np.cos(angle / 2)))  # Quaternion rotation 
    
    plane.position = origin
    return plane

def draw_axis():
    axis_vertices = np.array([
    [0, 0, 0], [1, 0, 0],  # X-axis (Red)
    [0, 0, 0], [0, 1, 0],  # Y-axis (Green)
    [0, 0, 0], [0, 0, 1]   # Z-axis (Blue)
    ], dtype=np.float32)

    axis_colors = np.array([
        [1, 0, 0], [1, 0, 0],  # Red for X
        [0, 1, 0], [0, 1, 0],  # Green for Y
        [0, 0, 1], [0, 0, 1]   # Blue for Z
    ], dtype=np.float32)

    # Create BufferGeometry for axes
    axis_geometry = BufferGeometry(
        attributes={
            'position': BufferAttribute(array=axis_vertices, normalized=False),
            'color': BufferAttribute(array=axis_colors, normalized=False),
        }
    )

    # Create material (Use vertex colors)
    axis_material = LineBasicMaterial(vertexColors='VertexColors', depthTest=True)

    # Create Line object
    axis_lines = LineSegments(geometry=axis_geometry, material=axis_material)
    return axis_lines

def generate_cube(width=1.0, height=1.0, depth=1.0):
    """Generate vertices and faces for a box."""
    vertices = np.array([
        [-width/2, -height/2, -depth/2],  # 0 - Back Bottom Left
        [ width/2, -height/2, -depth/2],  # 1 - Back Bottom Right
        [ width/2,  height/2, -depth/2],  # 2 - Back Top Right
        [-width/2,  height/2, -depth/2],  # 3 - Back Top Left
        [-width/2, -height/2,  depth/2],  # 4 - Front Bottom Left
        [ width/2, -height/2,  depth/2],  # 5 - Front Bottom Right
        [ width/2,  height/2,  depth/2],  # 6 - Front Top Right
        [-width/2,  height/2,  depth/2]   # 7 - Front Top Left
    ], dtype=np.float32)

    # Define 12 triangles (6 faces x 2 triangles per face)
    faces = np.array([
        [4, 5, 6], [4, 6, 7],  # Front (+Z)
        [1, 0, 3], [1, 3, 2],  # Back (-Z)
        [5, 1, 2], [5, 2, 6],  # Right (+X)
        [0, 4, 7], [0, 7, 3],  # Left (-X)
        [3, 7, 6], [3, 6, 2],  # Top (+Y)
        [0, 1, 5], [0, 5, 4]   # Bottom (-Y)
    ], dtype=np.uint32)
    
    # Define normals per triangle face (each normal repeats for 2 triangles per face)
    normals = np.array([
        [-0.577, -0.577, -0.577],  # 0 - Averaged from Back, Bottom, Left
        [ 0.577, -0.577, -0.577],  # 1 - Averaged from Back, Bottom, Right
        [ 0.577,  0.577, -0.577],  # 2 - Averaged from Back, Top, Right
        [-0.577,  0.577, -0.577],  # 3 - Averaged from Back, Top, Left
        [-0.577, -0.577,  0.577],  # 4 - Averaged from Front, Bottom, Left
        [ 0.577, -0.577,  0.577],  # 5 - Averaged from Front, Bottom, Right
        [ 0.577,  0.577,  0.577],  # 6 - Averaged from Front, Top, Right
        [-0.577,  0.577,  0.577]   # 7 - Averaged from Front, Top, Left
    ], dtype=np.float32)
    return vertices, faces, normals

def draw_cube(width=1.0, height=1.0, depth=1.0, color='green', position=[0, 0, 0]):
    """Draw a cube with a given size and color."""
    vertices, faces, normals = generate_cube(width, height, depth)
    geometry = BufferGeometry(
        attributes={
            'position': BufferAttribute(array=vertices.copy(), normalized=False),
            'normal': BufferAttribute(array=normals.copy(), normalized=True),
        },
        index=BufferAttribute(array=faces.copy().ravel(), normalized=False)
    )
    material = MeshStandardMaterial(
        color=color,  # RGB color of the material
        flatShading=True,
    )
    cube = Mesh(geometry=geometry, material=material)
    cube.castShadow = True
    
    cube.geometry.attributes['position'].array += np.array(position, dtype=np.float32)
    return cube

def update_mesh(bodies):
    for body in bodies:
        body.mesh.position = body.currPos
        body.mesh.quaternion = body.currRot


In [ ]:
def obb_collision(body1, body2):
    """Check if two rotated cubes (OBBs) collide using Separating Axis Theorem (SAT)."""
    v1 = body1.get_world_vertices()
    v2 = body2.get_world_vertices()

    # Compute min/max from transformed vertices
    min1, max1 = np.min(v1, axis=0), np.max(v1, axis=0)
    min2, max2 = np.min(v2, axis=0), np.max(v2, axis=0)

    return (
        (min1[0] <= max2[0] and max1[0] >= min2[0]) and  # X-axis
        (min1[1] <= max2[1] and max1[1] >= min2[1]) and  # Y-axis
        (min1[2] <= max2[2] and max1[2] >= min2[2])      # Z-axis
    )
    
def compute_obb_contact(body1, body2):
    """Compute contact normal and penetration depth for OBB collision."""
    v1 = body1.get_world_vertices()
    v2 = body2.get_world_vertices()

    # Compute min/max from transformed vertices
    min1, max1 = np.min(v1, axis=0), np.max(v1, axis=0)
    min2, max2 = np.min(v2, axis=0), np.max(v2, axis=0)

    # Compute overlap along each axis
    overlap_x = min(max1[0] - min2[0], max2[0] - min1[0])
    overlap_y = min(max1[1] - min2[1], max2[1] - min1[1])
    overlap_z = min(max1[2] - min2[2], max2[2] - min1[2])

    # Find the smallest penetration axis
    min_overlap = min(overlap_x, overlap_y, overlap_z)

    # Determine contact normal based on penetration axis
    contact_normal = np.zeros(3)
    if min_overlap == overlap_x:
        contact_normal[0] = 1 if (min1[0] < min2[0]) else -1
    elif min_overlap == overlap_y:
        contact_normal[1] = 1 if (min1[1] < min2[1]) else -1
    else:
        contact_normal[2] = 1 if (min1[2] < min2[2]) else -1

    return contact_normal, min_overlap
    

In [21]:
class RigidBody:
    def __init__(self, position=[0, 0, 0], vertices=None, faces=None, mass=1.0):
        
        # Mesh configuration
        # self.vertices = vertices
        # self.faces = faces
        # if self.vertices is None or self.faces is None:
        #     self.vertices, self.faces = generate_box()

        # self.geometry = BufferGeometry(
        #     attributes={
        #         'position': BufferAttribute(self.vertices, normalized=False),
        #     },
        #     index=BufferAttribute(self.faces.ravel(), normalized=False)
        # )
        # self.material = MeshStandardMaterial(color='black', wireframe=False)
        # self.mesh = Mesh(geometry=self.geometry, material=self.material)
        self.mesh = draw_cube(position=position)
        
        # Mass properties
        self.mass = mass
        self.invMass = 1.0 / mass if mass != 0 else 0
        
        self.inertia = np.eye(3) / 6 * self.mass  # Moment of inertia for a cube
        self.invInertia = np.linalg.inv(self.inertia)
        
        # Linear state
        self.currPos = np.array(self.mesh.position) 
        self.prevPos = np.array(self.mesh.position)
        self.currVel = np.zeros_like(self.currPos)
        self.prevVel = np.zeros_like(self.prevPos)
        
        # Angular state
        self.currRot = np.array([1.0, 0.0, 0.0, 0.0])  # Quaternion [w, x, y, z]
        self.prevRot = np.array([1.0, 0.0, 0.0, 0.0])
        self.currAngVel = np.zeros(3)
        self.prevAngVel = np.zeros(3)
        
        # Forces and torques
        self.force = np.zeros(3)
        self.torque = np.zeros(3)
        
    def get_world_vertices(self,):
        vertices = self.mesh.geometry.attributes['position'].array

        # Convert quaternion to rotation matrix
        q = self.currRot
        rot_matrix = np.array([
            [1 - 2 * (q.y ** 2 + q.z ** 2), 2 * (q.x * q.y - q.z * q.w), 2 * (q.x * q.z + q.y * q.w)],
            [2 * (q.x * q.y + q.z * q.w), 1 - 2 * (q.x ** 2 + q.z ** 2), 2 * (q.y * q.z - q.x * q.w)],
            [2 * (q.x * q.z - q.y * q.w), 2 * (q.y * q.z + q.x * q.w), 1 - 2 * (q.x ** 2 + q.y ** 2)]
        ])

        # Rotate and translate vertices to world space
        world_vertices = (vertices @ rot_matrix.T) + self.currPos
        return world_vertices
    

In [29]:
def quaternion_multiply(quaternion1, quaternion0):
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                     x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                     -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                     x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)

def BodyPairCorrection(body1, body2, correction, r1, r2, compliance, h, constraints_type):  
    
    if constraints_type == 'positional':
        # Positional Constraints    
        # correction is a linear vector
        c = np.linalg.norm(correction)
        if c < 1e-5:
            return 
        normal = correction / c    
        
        w1 = body1.invMass + np.cross(r1, normal)[None] @ body1.invInertia @ np.cross(r1, normal) # (2)
        w2 = body2.invMass + np.cross(r2, normal)[None] @ body2.invInertia @ np.cross(r2, normal) # (3)
        
        init_lambda = 0
        alpha_tilde = compliance / h**2
        dlambda = -c - init_lambda*alpha_tilde  / (w1 + w2 + alpha_tilde) # (4)
        ## Positional Impulse
        p = dlambda * normal 
        
        ### (6), (7)
        body1.currPos += body1.invMass * p
        body2.currPos -= body2.invMass * p
        
        angle_correction1 = body1.invInertia @ np.cross(r1, p)
        angle_correction2 = body2.invInertia @ np.cross(r2, p)
        ### (8)
        body1.currRot += 0.5 * quaternion_multiply(np.array([0, angle_correction1[0], angle_correction1[1], angle_correction1[2]]), body1.currRot)
        body1.currRot /= np.linalg.norm(body1.currRot)
        ### (9)
        body2.currRot -= 0.5 * quaternion_multiply(np.array([0, angle_correction2[0], angle_correction2[1], angle_correction2[2]]), body2.currRot)
        body2.currRot /= np.linalg.norm(body2.currRot)
    
    elif constraints_type == 'rotational':
        # Rotational Constraints
        # correction is an angular velocity vector
        # r1, r2 are None for this case
        angle = np.linalg.norm(correction)
        if angle < 1e-5:
            return
        axis = correction / angle
        
        w1 = axis[None] @ body1.invInertia @ axis # (11) 
        w2 = axis[None] @ body2.invInertia @ axis # (12)
        
        init_lambda = 0
        alpha_tilde = compliance / h**2
        dlambda = -angle - init_lambda*alpha_tilde  / (w1 + w2 + alpha_tilde) # (13)
        ## Rotational Impulse
        p = dlambda * axis
        
        angle_correction1 = body1.invInertia @ p
        angle_correction2 = body2.invInertia @ p
        ### (15)
        body1.currRot += 0.5 * quaternion_multiply(np.array([0, angle_correction1[0], angle_correction1[1], angle_correction1[2]]), body1.currRot)
        body1.currRot /= np.linalg.norm(body1.currRot)
        ### (16)
        body2.currRot -= 0.5 * quaternion_multiply(np.array([0, angle_correction2[0], angle_correction2[1], angle_correction2[2]]), body2.currRot)
        body2.currRot /= np.linalg.norm(body2.currRot)
    

def collect_collisions(bodies, plane):
    collsions = []
    for i in range(len(bodies)):
        # Check collision with other bodies
        for j in range(i+1, len(bodies)):
            if obb_collision(bodies[i], bodies[j]):
                collsions.append((bodies[i], bodies[j]))

        # Check collision with the ground plane
        if np.any(bodies[i].get_world_vertices()[:, 1] < 0):
            collsions.append((bodies[i], plane))

    return collsions

def check_contacts(collisions):
    contacts = []
    for (body1, body2) in collisions:
        if body2.isinstance(RigidBody):
            contact_normal, depth = compute_obb_contact(body1, body2)
            contacts.append((body1, body2, contact_normal, depth))
        
        indices = np.where(body.vertices[:, 1] < 0)[0]
        contact_depth = body.vertices[..., 1]
        contact_normal = plane.normal
        contacts.extend([(body, i, contact_normal, contact_depth[i]) for i in indices])
        
    return contacts

def integrate(body, gravity, h):
    # Update Linear state
    body.prevPos = body.currPos
    body.currVel += h * gravity * body.invMass
    body.currPos += h * body.currVel
    
    # Update Angular state
    body.prevRot = body.currRot
    body.currAngVel += h * body.invInertia @ (body.torque - np.cross(body.currAngVel, body.inertia @ body.currAngVel))
    body.currRot += h * 0.5 * quaternion_multiply(np.array([0, body.currAngVel[0], body.currAngVel[1], body.currAngVel[2]]), body.currRot)
    body.currRot /= np.linalg.norm(body.currRot)
    
def solve_positions(constraints):
    for constraint in constraints:
        constraint.solve()

def solve_contacts(contacts, h):
    for contact in contacts:
        body1, body2, normal, depth, r1, r2 = contact
        correction = depth * normal
        r1 = body.vertices[idx] - body.currPos
        r2 = None
        BodyPairCorrection(body, plane, correction, r1, r2, compliance=0.0, h=h, constraints_type='positional')
        
def update_velocities(body, h):
    body.currVel = (body.currPos - body.prevPos)/h
    body.currAngVel = (body.currRot - body.prevRot)/h

def solve_velocities():
    pass    


In [ ]:
# Rendering setup
width, height = 800, 600  # Smaller dimensions for 2D
aspect = width / height

scene = Scene(background='black')  # Background color of the scene
camera = PerspectiveCamera(
    position=[5, 5, 10], 
    lookAt=[0, 0, 0],
    aspect=aspect
    )

ambient_light = AmbientLight(color='white', intensity=1.0)  # Soft overall light
spot_light = SpotLight(
    color='white',
    intensity=3.0,        # Increase brightness
    position=[0, 5, 0],   # Place the light above the cube
    distance=10,          # Light range (covers cube)
    angle=0.5,            # Cone angle (medium focus)
    penumbra=0.3,         # Soft shadow edges
    decay=2,              # Realistic falloff
)
spot_light.castShadow = True  # Enable shadow casting
scene.add([ambient_light, spot_light])

bodies = [RigidBody(position=[0, 2, 0])]
plane = draw_plane(normal=[0, 1, 0], size=10, color='gray', opacity=0.5)
scene.add([body.mesh for body in bodies] + [plane])



controller = OrbitControls(controlling=camera, enableRotate=True)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
renderer.shadowMap.enabled = True
renderer.shadowMap.type = 'PCFSoftShadowMap'

display(widgets.VBox([renderer,]))

AttributeError: 'BufferGeometry' object has no attribute 'boundingBox'

In [26]:
# World Setup
constraints = []

# Simulation parameters
gravity = np.array([0, -9.81, 0])
time_step = 1/30
substeps = 10
h = time_step/substeps
while True:
    ## Gather Collisions Candidates
    collisions = collect_collisions(bodies, plane)
    
    for _ in range(substeps):
        ## Check Actual Collisions
        contacts = check_contacts(collisions)
        
        for body in bodies:
            integrate(body, gravity, h) # update position and orientation with external forces and torques
            
        ## Solve Constraints
        for constraint in constraints:
            solve_positions()
        ## Solve Collisons
        solve_contacts(contacts)
        
        ## Updata Velocities with solved positions
        for body in bodies:
            update_velocities()
            
        ## Solve Velocities  - Damping and Friction
        solve_velocities()
    
    update_mesh(bodies)
    time.sleep(0.01)

AttributeError: 'RigidBody' object has no attribute 'vertices'